In [36]:
import json
import multiprocessing as mp
import os
import time
import uuid

import random

import boto3
from PIL import Image
import sagemaker
from sagemaker import Session
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role


sess = Session()
bucket = 'waymo-open-dataset1'  # SageMaker will use this bucket to store data, script and model checkpoints

s3 = boto3.client('s3')

In [37]:
work_dir = '/home/ec2-user/SageMaker'
dataset_prefix = 'waymo_data'
data_dir = work_dir + '/' + dataset_prefix

# locations used for local testing
local_dataset_cache = work_dir + '/waymo-tmp'
local_checkpoint_location = work_dir + '/waymo-checkpoints'

In [15]:
%%time

# Download images. This took 12min on a ml.g4dn.16xlarge instance in eu-west-1 region
! aws s3 cp s3://waymo-open-dataset1/image_0/ /home/ec2-user/SageMaker/waymo_data/image_0 --recursive

download: s3://waymo-open-dataset1/image_0/000000000000004.jpg to ../waymo_data/image_0/000000000000004.jpg
download: s3://waymo-open-dataset1/image_0/000000000000001.jpg to ../waymo_data/image_0/000000000000001.jpg
download: s3://waymo-open-dataset1/image_0/000000000000007.jpg to ../waymo_data/image_0/000000000000007.jpg
download: s3://waymo-open-dataset1/image_0/000000000000010.jpg to ../waymo_data/image_0/000000000000010.jpg
download: s3://waymo-open-dataset1/image_0/000000000000002.jpg to ../waymo_data/image_0/000000000000002.jpg
download: s3://waymo-open-dataset1/image_0/000000000000006.jpg to ../waymo_data/image_0/000000000000006.jpg
download: s3://waymo-open-dataset1/image_0/000000000000009.jpg to ../waymo_data/image_0/000000000000009.jpg
download: s3://waymo-open-dataset1/image_0/000000000000005.jpg to ../waymo_data/image_0/000000000000005.jpg
download: s3://waymo-open-dataset1/image_0/000000000000000.jpg to ../waymo_data/image_0/000000000000000.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000000085.jpg to ../waymo_data/image_0/000000000000085.jpg
download: s3://waymo-open-dataset1/image_0/000000000000081.jpg to ../waymo_data/image_0/000000000000081.jpg
download: s3://waymo-open-dataset1/image_0/000000000000078.jpg to ../waymo_data/image_0/000000000000078.jpg
download: s3://waymo-open-dataset1/image_0/000000000000086.jpg to ../waymo_data/image_0/000000000000086.jpg
download: s3://waymo-open-dataset1/image_0/000000000000082.jpg to ../waymo_data/image_0/000000000000082.jpg
download: s3://waymo-open-dataset1/image_0/000000000000084.jpg to ../waymo_data/image_0/000000000000084.jpg
download: s3://waymo-open-dataset1/image_0/000000000000083.jpg to ../waymo_data/image_0/000000000000083.jpg
download: s3://waymo-open-dataset1/image_0/000000000000080.jpg to ../waymo_data/image_0/000000000000080.jpg
download: s3://waymo-open-dataset1/image_0/000000000000088.jpg to ../waymo_data/image_0/000000000000088.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000000157.jpg to ../waymo_data/image_0/000000000000157.jpg
download: s3://waymo-open-dataset1/image_0/000000000000153.jpg to ../waymo_data/image_0/000000000000153.jpg
download: s3://waymo-open-dataset1/image_0/000000000000156.jpg to ../waymo_data/image_0/000000000000156.jpg
download: s3://waymo-open-dataset1/image_0/000000000000160.jpg to ../waymo_data/image_0/000000000000160.jpg
download: s3://waymo-open-dataset1/image_0/000000000000161.jpg to ../waymo_data/image_0/000000000000161.jpg
download: s3://waymo-open-dataset1/image_0/000000000000163.jpg to ../waymo_data/image_0/000000000000163.jpg
download: s3://waymo-open-dataset1/image_0/000000000000164.jpg to ../waymo_data/image_0/000000000000164.jpg
download: s3://waymo-open-dataset1/image_0/000000000000162.jpg to ../waymo_data/image_0/000000000000162.jpg
download: s3://waymo-open-dataset1/image_0/000000000000166.jpg to ../waymo_data/image_0/000000000000166.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000000239.jpg to ../waymo_data/image_0/000000000000239.jpg
download: s3://waymo-open-dataset1/image_0/000000000000235.jpg to ../waymo_data/image_0/000000000000235.jpg
download: s3://waymo-open-dataset1/image_0/000000000000233.jpg to ../waymo_data/image_0/000000000000233.jpg
download: s3://waymo-open-dataset1/image_0/000000000000237.jpg to ../waymo_data/image_0/000000000000237.jpg
download: s3://waymo-open-dataset1/image_0/000000000000238.jpg to ../waymo_data/image_0/000000000000238.jpg
download: s3://waymo-open-dataset1/image_0/000000000000242.jpg to ../waymo_data/image_0/000000000000242.jpg
download: s3://waymo-open-dataset1/image_0/000000000000243.jpg to ../waymo_data/image_0/000000000000243.jpg
download: s3://waymo-open-dataset1/image_0/000000000000244.jpg to ../waymo_data/image_0/000000000000244.jpg
download: s3://waymo-open-dataset1/image_0/000000000000240.jpg to ../waymo_data/image_0/000000000000240.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000000318.jpg to ../waymo_data/image_0/000000000000318.jpg
download: s3://waymo-open-dataset1/image_0/000000000000314.jpg to ../waymo_data/image_0/000000000000314.jpg
download: s3://waymo-open-dataset1/image_0/000000000000307.jpg to ../waymo_data/image_0/000000000000307.jpg
download: s3://waymo-open-dataset1/image_0/000000000000313.jpg to ../waymo_data/image_0/000000000000313.jpg
download: s3://waymo-open-dataset1/image_0/000000000000321.jpg to ../waymo_data/image_0/000000000000321.jpg
download: s3://waymo-open-dataset1/image_0/000000000000317.jpg to ../waymo_data/image_0/000000000000317.jpg
download: s3://waymo-open-dataset1/image_0/000000000000316.jpg to ../waymo_data/image_0/000000000000316.jpg
download: s3://waymo-open-dataset1/image_0/000000000000315.jpg to ../waymo_data/image_0/000000000000315.jpg
download: s3://waymo-open-dataset1/image_0/000000000000320.jpg to ../waymo_data/image_0/000000000000320.jpg
download: s3://waymo-open-da

...

download: s3://waymo-open-dataset1/image_0/000000000010324.jpg to ../waymo_data/image_0/000000000010324.jpg
download: s3://waymo-open-dataset1/image_0/000000000010316.jpg to ../waymo_data/image_0/000000000010316.jpg
download: s3://waymo-open-dataset1/image_0/000000000010329.jpg to ../waymo_data/image_0/000000000010329.jpg
download: s3://waymo-open-dataset1/image_0/000000000010326.jpg to ../waymo_data/image_0/000000000010326.jpg
download: s3://waymo-open-dataset1/image_0/000000000010330.jpg to ../waymo_data/image_0/000000000010330.jpg
download: s3://waymo-open-dataset1/image_0/000000000010331.jpg to ../waymo_data/image_0/000000000010331.jpg
download: s3://waymo-open-dataset1/image_0/000000000010332.jpg to ../waymo_data/image_0/000000000010332.jpg
download: s3://waymo-open-dataset1/image_0/000000000010328.jpg to ../waymo_data/image_0/000000000010328.jpg
download: s3://waymo-open-dataset1/image_0/000000000010333.jpg to ../waymo_data/image_0/000000000010333.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000010393.jpg to ../waymo_data/image_0/000000000010393.jpg
download: s3://waymo-open-dataset1/image_0/000000000010399.jpg to ../waymo_data/image_0/000000000010399.jpg
download: s3://waymo-open-dataset1/image_0/000000000010405.jpg to ../waymo_data/image_0/000000000010405.jpg
download: s3://waymo-open-dataset1/image_0/000000000010406.jpg to ../waymo_data/image_0/000000000010406.jpg
download: s3://waymo-open-dataset1/image_0/000000000010404.jpg to ../waymo_data/image_0/000000000010404.jpg
download: s3://waymo-open-dataset1/image_0/000000000010403.jpg to ../waymo_data/image_0/000000000010403.jpg
download: s3://waymo-open-dataset1/image_0/000000000010408.jpg to ../waymo_data/image_0/000000000010408.jpg
download: s3://waymo-open-dataset1/image_0/000000000010407.jpg to ../waymo_data/image_0/000000000010407.jpg
download: s3://waymo-open-dataset1/image_0/000000000010416.jpg to ../waymo_data/image_0/000000000010416.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000010479.jpg to ../waymo_data/image_0/000000000010479.jpg
download: s3://waymo-open-dataset1/image_0/000000000010482.jpg to ../waymo_data/image_0/000000000010482.jpg
download: s3://waymo-open-dataset1/image_0/000000000010480.jpg to ../waymo_data/image_0/000000000010480.jpg
download: s3://waymo-open-dataset1/image_0/000000000010484.jpg to ../waymo_data/image_0/000000000010484.jpg
download: s3://waymo-open-dataset1/image_0/000000000010477.jpg to ../waymo_data/image_0/000000000010477.jpg
download: s3://waymo-open-dataset1/image_0/000000000010487.jpg to ../waymo_data/image_0/000000000010487.jpg
download: s3://waymo-open-dataset1/image_0/000000000010481.jpg to ../waymo_data/image_0/000000000010481.jpg
download: s3://waymo-open-dataset1/image_0/000000000010486.jpg to ../waymo_data/image_0/000000000010486.jpg
download: s3://waymo-open-dataset1/image_0/000000000010483.jpg to ../waymo_data/image_0/000000000010483.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000010558.jpg to ../waymo_data/image_0/000000000010558.jpg
download: s3://waymo-open-dataset1/image_0/000000000010560.jpg to ../waymo_data/image_0/000000000010560.jpg
download: s3://waymo-open-dataset1/image_0/000000000010561.jpg to ../waymo_data/image_0/000000000010561.jpg
download: s3://waymo-open-dataset1/image_0/000000000010557.jpg to ../waymo_data/image_0/000000000010557.jpg
download: s3://waymo-open-dataset1/image_0/000000000010556.jpg to ../waymo_data/image_0/000000000010556.jpg
download: s3://waymo-open-dataset1/image_0/000000000010562.jpg to ../waymo_data/image_0/000000000010562.jpg
download: s3://waymo-open-dataset1/image_0/000000000010563.jpg to ../waymo_data/image_0/000000000010563.jpg
download: s3://waymo-open-dataset1/image_0/000000000010568.jpg to ../waymo_data/image_0/000000000010568.jpg
download: s3://waymo-open-dataset1/image_0/000000000010565.jpg to ../waymo_data/image_0/000000000010565.jpg
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/image_0/000000000010634.jpg to ../waymo_data/image_0/000000000010634.jpg
download: s3://waymo-open-dataset1/image_0/000000000010639.jpg to ../waymo_data/image_0/000000000010639.jpg
download: s3://waymo-open-dataset1/image_0/000000000010638.jpg to ../waymo_data/image_0/000000000010638.jpg
download: s3://waymo-open-dataset1/image_0/000000000010636.jpg to ../waymo_data/image_0/000000000010636.jpg
download: s3://waymo-open-dataset1/image_0/000000000010635.jpg to ../waymo_data/image_0/000000000010635.jpg
download: s3://waymo-open-dataset1/image_0/000000000010640.jpg to ../waymo_data/image_0/000000000010640.jpg
download: s3://waymo-open-dataset1/image_0/000000000010642.jpg to ../waymo_data/image_0/000000000010642.jpg
download: s3://waymo-open-dataset1/image_0/000000000010644.jpg to ../waymo_data/image_0/000000000010644.jpg
download: s3://waymo-open-dataset1/image_0/000000000010641.jpg to ../waymo_data/image_0/000000000010641.jpg
download: s3://waymo-open-da

In [16]:
%%time

# Download images. This took 12min on a ml.g4dn.16xlarge instance in eu-west-1 region
! aws s3 cp s3://waymo-open-dataset1/label_0/ /home/ec2-user/SageMaker/waymo_data/label_0 --recursive

download: s3://waymo-open-dataset1/label_0/000000000000000.txt to ../waymo_data/label_0/000000000000000.txt
download: s3://waymo-open-dataset1/label_0/000000000000001.txt to ../waymo_data/label_0/000000000000001.txt
download: s3://waymo-open-dataset1/label_0/000000000000003.txt to ../waymo_data/label_0/000000000000003.txt
download: s3://waymo-open-dataset1/label_0/000000000000004.txt to ../waymo_data/label_0/000000000000004.txt
download: s3://waymo-open-dataset1/label_0/000000000000006.txt to ../waymo_data/label_0/000000000000006.txt
download: s3://waymo-open-dataset1/label_0/000000000000002.txt to ../waymo_data/label_0/000000000000002.txt
download: s3://waymo-open-dataset1/label_0/000000000000005.txt to ../waymo_data/label_0/000000000000005.txt
download: s3://waymo-open-dataset1/label_0/000000000000008.txt to ../waymo_data/label_0/000000000000008.txt
download: s3://waymo-open-dataset1/label_0/000000000000007.txt to ../waymo_data/label_0/000000000000007.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000000080.txt to ../waymo_data/label_0/000000000000080.txt
download: s3://waymo-open-dataset1/label_0/000000000000081.txt to ../waymo_data/label_0/000000000000081.txt
download: s3://waymo-open-dataset1/label_0/000000000000079.txt to ../waymo_data/label_0/000000000000079.txt
download: s3://waymo-open-dataset1/label_0/000000000000083.txt to ../waymo_data/label_0/000000000000083.txt
download: s3://waymo-open-dataset1/label_0/000000000000078.txt to ../waymo_data/label_0/000000000000078.txt
download: s3://waymo-open-dataset1/label_0/000000000000077.txt to ../waymo_data/label_0/000000000000077.txt
download: s3://waymo-open-dataset1/label_0/000000000000084.txt to ../waymo_data/label_0/000000000000084.txt
download: s3://waymo-open-dataset1/label_0/000000000000082.txt to ../waymo_data/label_0/000000000000082.txt
download: s3://waymo-open-dataset1/label_0/000000000000075.txt to ../waymo_data/label_0/000000000000075.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000000157.txt to ../waymo_data/label_0/000000000000157.txt
download: s3://waymo-open-dataset1/label_0/000000000000149.txt to ../waymo_data/label_0/000000000000149.txt
download: s3://waymo-open-dataset1/label_0/000000000000159.txt to ../waymo_data/label_0/000000000000159.txt
download: s3://waymo-open-dataset1/label_0/000000000000158.txt to ../waymo_data/label_0/000000000000158.txt
download: s3://waymo-open-dataset1/label_0/000000000000155.txt to ../waymo_data/label_0/000000000000155.txt
download: s3://waymo-open-dataset1/label_0/000000000000161.txt to ../waymo_data/label_0/000000000000161.txt
download: s3://waymo-open-dataset1/label_0/000000000000156.txt to ../waymo_data/label_0/000000000000156.txt
download: s3://waymo-open-dataset1/label_0/000000000000164.txt to ../waymo_data/label_0/000000000000164.txt
download: s3://waymo-open-dataset1/label_0/000000000000162.txt to ../waymo_data/label_0/000000000000162.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000000220.txt to ../waymo_data/label_0/000000000000220.txt
download: s3://waymo-open-dataset1/label_0/000000000000236.txt to ../waymo_data/label_0/000000000000236.txt
download: s3://waymo-open-dataset1/label_0/000000000000237.txt to ../waymo_data/label_0/000000000000237.txt
download: s3://waymo-open-dataset1/label_0/000000000000234.txt to ../waymo_data/label_0/000000000000234.txt
download: s3://waymo-open-dataset1/label_0/000000000000235.txt to ../waymo_data/label_0/000000000000235.txt
download: s3://waymo-open-dataset1/label_0/000000000000242.txt to ../waymo_data/label_0/000000000000242.txt
download: s3://waymo-open-dataset1/label_0/000000000000238.txt to ../waymo_data/label_0/000000000000238.txt
download: s3://waymo-open-dataset1/label_0/000000000000239.txt to ../waymo_data/label_0/000000000000239.txt
download: s3://waymo-open-dataset1/label_0/000000000000231.txt to ../waymo_data/label_0/000000000000231.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000000319.txt to ../waymo_data/label_0/000000000000319.txt
download: s3://waymo-open-dataset1/label_0/000000000000324.txt to ../waymo_data/label_0/000000000000324.txt
download: s3://waymo-open-dataset1/label_0/000000000000325.txt to ../waymo_data/label_0/000000000000325.txt
download: s3://waymo-open-dataset1/label_0/000000000000321.txt to ../waymo_data/label_0/000000000000321.txt
download: s3://waymo-open-dataset1/label_0/000000000000316.txt to ../waymo_data/label_0/000000000000316.txt
download: s3://waymo-open-dataset1/label_0/000000000000320.txt to ../waymo_data/label_0/000000000000320.txt
download: s3://waymo-open-dataset1/label_0/000000000000323.txt to ../waymo_data/label_0/000000000000323.txt
download: s3://waymo-open-dataset1/label_0/000000000000326.txt to ../waymo_data/label_0/000000000000326.txt
download: s3://waymo-open-dataset1/label_0/000000000000327.txt to ../waymo_data/label_0/000000000000327.txt
download: s3://waymo-open-da

...

download: s3://waymo-open-dataset1/label_0/000000000010363.txt to ../waymo_data/label_0/000000000010363.txt
download: s3://waymo-open-dataset1/label_0/000000000010364.txt to ../waymo_data/label_0/000000000010364.txt
download: s3://waymo-open-dataset1/label_0/000000000010356.txt to ../waymo_data/label_0/000000000010356.txt
download: s3://waymo-open-dataset1/label_0/000000000010366.txt to ../waymo_data/label_0/000000000010366.txt
download: s3://waymo-open-dataset1/label_0/000000000010367.txt to ../waymo_data/label_0/000000000010367.txt
download: s3://waymo-open-dataset1/label_0/000000000010371.txt to ../waymo_data/label_0/000000000010371.txt
download: s3://waymo-open-dataset1/label_0/000000000010373.txt to ../waymo_data/label_0/000000000010373.txt
download: s3://waymo-open-dataset1/label_0/000000000010368.txt to ../waymo_data/label_0/000000000010368.txt
download: s3://waymo-open-dataset1/label_0/000000000010372.txt to ../waymo_data/label_0/000000000010372.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000010440.txt to ../waymo_data/label_0/000000000010440.txt
download: s3://waymo-open-dataset1/label_0/000000000010448.txt to ../waymo_data/label_0/000000000010448.txt
download: s3://waymo-open-dataset1/label_0/000000000010451.txt to ../waymo_data/label_0/000000000010451.txt
download: s3://waymo-open-dataset1/label_0/000000000010445.txt to ../waymo_data/label_0/000000000010445.txt
download: s3://waymo-open-dataset1/label_0/000000000010431.txt to ../waymo_data/label_0/000000000010431.txt
download: s3://waymo-open-dataset1/label_0/000000000010449.txt to ../waymo_data/label_0/000000000010449.txt
download: s3://waymo-open-dataset1/label_0/000000000010453.txt to ../waymo_data/label_0/000000000010453.txt
download: s3://waymo-open-dataset1/label_0/000000000010450.txt to ../waymo_data/label_0/000000000010450.txt
download: s3://waymo-open-dataset1/label_0/000000000010456.txt to ../waymo_data/label_0/000000000010456.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000010528.txt to ../waymo_data/label_0/000000000010528.txt
download: s3://waymo-open-dataset1/label_0/000000000010530.txt to ../waymo_data/label_0/000000000010530.txt
download: s3://waymo-open-dataset1/label_0/000000000010525.txt to ../waymo_data/label_0/000000000010525.txt
download: s3://waymo-open-dataset1/label_0/000000000010523.txt to ../waymo_data/label_0/000000000010523.txt
download: s3://waymo-open-dataset1/label_0/000000000010524.txt to ../waymo_data/label_0/000000000010524.txt
download: s3://waymo-open-dataset1/label_0/000000000010520.txt to ../waymo_data/label_0/000000000010520.txt
download: s3://waymo-open-dataset1/label_0/000000000010531.txt to ../waymo_data/label_0/000000000010531.txt
download: s3://waymo-open-dataset1/label_0/000000000010529.txt to ../waymo_data/label_0/000000000010529.txt
download: s3://waymo-open-dataset1/label_0/000000000010534.txt to ../waymo_data/label_0/000000000010534.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000010603.txt to ../waymo_data/label_0/000000000010603.txt
download: s3://waymo-open-dataset1/label_0/000000000010601.txt to ../waymo_data/label_0/000000000010601.txt
download: s3://waymo-open-dataset1/label_0/000000000010602.txt to ../waymo_data/label_0/000000000010602.txt
download: s3://waymo-open-dataset1/label_0/000000000010600.txt to ../waymo_data/label_0/000000000010600.txt
download: s3://waymo-open-dataset1/label_0/000000000010606.txt to ../waymo_data/label_0/000000000010606.txt
download: s3://waymo-open-dataset1/label_0/000000000010608.txt to ../waymo_data/label_0/000000000010608.txt
download: s3://waymo-open-dataset1/label_0/000000000010605.txt to ../waymo_data/label_0/000000000010605.txt
download: s3://waymo-open-dataset1/label_0/000000000010607.txt to ../waymo_data/label_0/000000000010607.txt
download: s3://waymo-open-dataset1/label_0/000000000010611.txt to ../waymo_data/label_0/000000000010611.txt
download: s3://waymo-open-da

download: s3://waymo-open-dataset1/label_0/000000000010681.txt to ../waymo_data/label_0/000000000010681.txt
download: s3://waymo-open-dataset1/label_0/000000000010685.txt to ../waymo_data/label_0/000000000010685.txt
download: s3://waymo-open-dataset1/label_0/000000000010683.txt to ../waymo_data/label_0/000000000010683.txt
download: s3://waymo-open-dataset1/label_0/000000000010684.txt to ../waymo_data/label_0/000000000010684.txt
download: s3://waymo-open-dataset1/label_0/000000000010672.txt to ../waymo_data/label_0/000000000010672.txt
download: s3://waymo-open-dataset1/label_0/000000000010682.txt to ../waymo_data/label_0/000000000010682.txt
download: s3://waymo-open-dataset1/label_0/000000000010688.txt to ../waymo_data/label_0/000000000010688.txt
download: s3://waymo-open-dataset1/label_0/000000000010686.txt to ../waymo_data/label_0/000000000010686.txt
download: s3://waymo-open-dataset1/label_0/000000000010690.txt to ../waymo_data/label_0/000000000010690.txt
download: s3://waymo-open-da

In [23]:
manifest = {}

# classes
# images
# labels

imgs = list(sorted(os.listdir(os.path.join(data_dir, "image_0"))))
labels = list(sorted(os.listdir(os.path.join(data_dir, "label_0"))))


#ADDING SAFETY MECHANISM TO ENSURE THAT ONLY IDS THAT ARE PRESENT IN BOTH IMAGES AND LABELS ARE USED
#--> OTHERWISE IINDEX OUT OF BOUNDS ERROR

imgs_split = [idx.split('.')[0] for idx in imgs]
labels_split = [idx.split('.')[0] for idx in labels]
usable_ids = list(set(imgs_split).intersection(set(labels_split)))

imgs = [idx for idx in imgs if idx.split('.')[0] in usable_ids]
labels = [idx for idx in labels if idx.split('.')[0] in usable_ids]

temp_class_list = [] # for getting all available classes
for i in imgs:
    img_id = i.split('.')[0]
    label_name = i.replace('.jpg', '.txt')
    img_path_local = os.path.join(data_dir, "image_0", i)
    label_path_local = os.path.join(data_dir, "label_0", label_name)
    
    with open(label_path_local) as file:
        lines = [line.rstrip().split(' ')[0] for line in file]
    temp_class_list.extend(lines)
    
    img_path_manifest = os.path.join("image_0", i)
    label_path_manifest = os.path.join("label_0", label_name)
    
    manifest[img_id] = {}
    manifest[img_id]['image_name'] = i
    manifest[img_id]['label_name'] = label_name
    manifest[img_id]['img_path'] = img_path_manifest
    manifest[img_id]['label_path'] = label_path_manifest
    
    
# extract all available classes from data

classes = list(set(temp_class_list))

manifest['classes'] = classes             
            
print("Created a dataset manifest with {} records".format(len(manifest) -1))

Created a dataset manifest with 10564 records


In [75]:
manifest

{'000000000000000': {'image_name': '000000000000000.jpg',
  'label_name': '000000000000000.txt',
  'img_path': 'image_0/000000000000000.jpg',
  'label_path': 'label_0/000000000000000.txt'},
 '000000000000001': {'image_name': '000000000000001.jpg',
  'label_name': '000000000000001.txt',
  'img_path': 'image_0/000000000000001.jpg',
  'label_path': 'label_0/000000000000001.txt'},
 '000000000000002': {'image_name': '000000000000002.jpg',
  'label_name': '000000000000002.txt',
  'img_path': 'image_0/000000000000002.jpg',
  'label_path': 'label_0/000000000000002.txt'},
 '000000000000003': {'image_name': '000000000000003.jpg',
  'label_name': '000000000000003.txt',
  'img_path': 'image_0/000000000000003.jpg',
  'label_path': 'label_0/000000000000003.txt'},
 '000000000000004': {'image_name': '000000000000004.jpg',
  'label_name': '000000000000004.txt',
  'img_path': 'image_0/000000000000004.jpg',
  'label_path': 'label_0/000000000000004.txt'},
 '000000000000005': {'image_name': '00000000000000

In [43]:
with open('/home/ec2-user/SageMaker/waymo_data/manifest.json', 'w') as file:
    json.dump(manifest, file)

def send_images_to_s3(image_id):
    
    s3.upload_file(Filename=data_dir + '/' + manifest[image_id]['image_path'],
                   Bucket=bucket,
                   Key=manifest[image_id]['image_path'])
    
    s3.upload_file(Filename=data_dir + '/' + manifest[image_id]['label_path'],
                   Bucket=bucket,
                   Key=manifest[image_id]['label_path'])

%%time

with mp.Pool(mp.cpu_count()) as pool:
    
    pool.map(send_images_to_s3, manifest.keys())

In [38]:
# create a folder to cache dataset as it is downloaded by the dataset class
print('Local dataset cache created at {}'.format(local_dataset_cache))
print('Local checkpoints will be stored at {}'.format(local_checkpoint_location))

try:
    os.mkdir(local_dataset_cache)
    
except(FileExistsError):
    print('{} already exists'.format(local_dataset_cache))

try:
    os.mkdir(local_checkpoint_location)
    
except(FileExistsError):
    print('{} already exists'.format(local_checkpoint_location))

Local dataset cache created at /home/ec2-user/SageMaker/waymo-tmp
Local checkpoints will be stored at /home/ec2-user/SageMaker/waymo-checkpoints
/home/ec2-user/SageMaker/waymo-tmp already exists
/home/ec2-user/SageMaker/waymo-checkpoints already exists


In [59]:
!pip install cython cocotools matplotlib

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6492 sha256=50c95703c5479829c658eeffe9db8570fffef3c0df72c72a7645c9b02cb0077c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/79/25/c2/c4e1b7ace859c22249e4976dd110211b7ee3640b2d84edfc13
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp39-cp39-linux_x86_64.whl size=94110 sha256=8329c3175de2811a55551d904eea4d3530e2f1b11bf2c728f966792001e5509c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/29/98/97/6c7dca1f8e4c854e15a2676ac98ae3f46ec83ee031d827a5c8
Successfully built easydict pycocotools


In [51]:
!git clone https://github.com/pytorch/vision.git ./waymo_code/vision

Cloning into './waymo_code/vision'...
remote: Enumerating objects: 294703, done.
remote: Counting objects: 100% (4495/4495), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 294703 (delta 4232), reused 4431 (delta 4192), pack-reused 290208
Receiving objects: 100% (294703/294703), 591.24 MiB | 59.31 MiB/s, done.
Resolving deltas: 100% (270183/270183), done.


In [53]:
!cp ./waymo_code/vision/references/detection/utils.py ./waymo_code

In [56]:
!cp ./waymo_code/vision/references/detection/transforms.py ./waymo_code

In [55]:
!cp ./waymo_code/vision/references/detection/coco_eval.py ./waymo_code

In [57]:
!cp ./waymo_code/vision/references/detection/engine.py ./waymo_code

In [58]:
!cp ./waymo_code/vision/references/detection/coco_utils.py ./waymo_code

In [4]:
%%time

# test on 20 iterations. 
# This takes a few minutes. You can see instance activity live using htop or nividia-smi in instance terminal
! python waymo_code/train_s3.py --dataset $data_dir \
                            --cache $local_dataset_cache \
                            --checkpoint-dir $local_checkpoint_location \
                            --batch 2 \
                            --workers 4 \
                            --log-freq 20 \
                            --eval-size 10 \
                            --iterations 20 

starting training
Epoch: [0]  [   0/5032]  eta: 5:10:09  lr: 0.000000  loss: 3.1173 (3.1173)  loss_classifier: 2.0785 (2.0785)  loss_box_reg: 0.0396 (0.0396)  loss_objectness: 0.7096 (0.7096)  loss_rpn_box_reg: 0.2896 (0.2896)  time: 3.6982  data: 0.3808  max mem: 7423
Epoch: [0]  [  10/5032]  eta: 1:42:17  lr: 0.000001  loss: 3.0255 (3.0246)  loss_classifier: 1.9631 (1.9894)  loss_box_reg: 0.0488 (0.0583)  loss_objectness: 0.6953 (0.6996)  loss_rpn_box_reg: 0.2587 (0.2773)  time: 1.2221  data: 0.0348  max mem: 8522
^C
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/sagemaker-a2d2-segmentation-pytorch/waymo_code/train_s3.py", line 210, in <module>
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
  File "/home/ec2-user/SageMaker/sagemaker-a2d2-segmentation-pytorch/waymo_code/engine.py", line 31, in train_one_epoch
    loss_dict = model(images, targets)
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/torch/

In [5]:
data_dir

'/home/ec2-user/SageMaker/waymo_data'

## Launch in Sagemaker Training

In [39]:
client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'waymo_pytorch_swin'

ecr_image = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name)

print(ecr_image)

855905809023.dkr.ecr.us-east-1.amazonaws.com/waymo_pytorch_swin:latest


In [14]:
s3.upload_file(
    Filename=work_dir + '/waymo_data/' + 'manifest.json',
    Bucket=bucket,
    Key='metadata/manifest.json')
    
train_path = 's3://{}/'.format(bucket) + 'metadata/'
print('Training manifests sent to {}'.format(train_path))

Training manifests sent to s3://waymo-open-dataset1/metadata/


In [11]:
train_path = 's3://{}/'.format(bucket) + 'metadata/'
print('Training manifests sent to {}'.format(train_path))

Training manifests sent to s3://waymo-open-dataset1/metadata/


In [7]:
config = {
    'bucket': bucket,
    'cache': '/opt/ml/input/data/dataset',
    'epochs': 10,
    'batch': 2,
    'workers': 4
}

In [8]:
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3

In [39]:
get_execution_role()

'arn:aws:iam::855905809023:role/sagemakerrole'

In [13]:
job = PyTorch(
    entry_point='train_s3.py', 
    source_dir='waymo_code', 
    role=get_execution_role(), 
    image_uri=ecr_image,
    instance_count = 1,
    instance_type = 'ml.p3.2xlarge',
    base_job_name='waymo-single-GPU-training',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token),
    environment={"SMDEBUG_LOG_LEVEL":"off"},
    debugger_hook_config=False,
    disable_profiler=True
    )

In [29]:
job = PyTorch(
    entry_point='train_s3.py', 
    source_dir='waymo_code', 
    role=get_execution_role(), 
    framework_version='1.12.1',
    instance_count = 1,
    instance_type = 'ml.p3.2xlarge',
    base_job_name='waymo-single-GPU-training',
    py_version='py38',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token),
    environment={"SMDEBUG_LOG_LEVEL":"off"},
    debugger_hook_config=False,
    disable_profiler=True
    )

In [14]:
job.fit({'dataset': train_path}, wait=False)

INFO:sagemaker:Creating training-job with name: waymo-single-GPU-training-2023-02-08-14-34-25-739


In [32]:
!python test.py

  File "/home/ec2-user/SageMaker/sagemaker-a2d2-segmentation-pytorch/test.py", line 4
    !mkdir test
    ^
SyntaxError: invalid syntax


## Train single-host-multi-node

In [ ]:
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3

In [ ]:
config = {
    'bucket': bucket,
    'cache': '/opt/ml/input/data/dataset',
    'epochs': 10,
    'batch': 2,
    'workers': 4
}

In [27]:
job = PyTorch(
    entry_point='train_s3_dist.py', 
    source_dir='waymo_code', 
    role=get_execution_role(), 
    image_uri=ecr_image,
    instance_count = 1,
    instance_type = 'ml.p3.16xlarge',
    base_job_name='waymo-multi-GPU-training',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token),
    environment={"SMDEBUG_LOG_LEVEL":"off"},
    debugger_hook_config=False,
    disable_profiler=True,
    
    distribution={"pytorchddp": 
                  {"enabled": True
                  }}
    )

In [28]:
job.fit({'dataset': train_path}, wait=False)

INFO:sagemaker:Creating training-job with name: waymo-multi-GPU-training-2023-02-08-15-56-17-281


## Train multi-host

In [29]:
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3

In [30]:
config = {
    'bucket': bucket,
    'cache': '/opt/ml/input/data/dataset',
    'epochs': 10,
    'batch': 2,
    'workers': 4
}

In [33]:
job = PyTorch(
    entry_point='train_s3_dist.py', 
    source_dir='waymo_code', 
    role=get_execution_role(), 
    image_uri=ecr_image,
    instance_count = 2,
    instance_type = 'ml.p3.8xlarge',
    base_job_name='waymo-multi-GPU-training',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token),
    environment={"SMDEBUG_LOG_LEVEL":"off"},
    debugger_hook_config=False,
    disable_profiler=True,
    
    distribution={"pytorchddp": 
                  {"enabled": True
                  }}
    )

In [ ]:
job.fit({'dataset': train_path}, wait=False)

## Train multi-host using smddp as backend

In [40]:
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3

In [41]:
config = {
    'bucket': bucket,
    'cache': '/opt/ml/input/data/dataset',
    'epochs': 10,
    'batch': 2,
    'workers': 4
}

In [46]:
job = PyTorch(
    entry_point='train_s3_dist_smddp.py', 
    source_dir='waymo_code', 
    role=get_execution_role(), 
    image_uri=ecr_image,
    instance_count = 2,
    instance_type = 'ml.p3.16xlarge',
    base_job_name='waymo-multi-GPU-training-smddp',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token),
    environment={"SMDEBUG_LOG_LEVEL":"off"},
    debugger_hook_config=False,
    disable_profiler=True,
    
    distribution = {"smdistributed":{"dataparallel":{"enabled":True}}} 
    )

In [47]:
job.fit({'dataset': train_path}, wait=False)

INFO:sagemaker:Creating training-job with name: waymo-multi-GPU-training-smddp-2023-02-09-10-23-15-680
